In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
from os import listdir
from os.path import isfile, join
import seaborn as sns
from multiprocessing import Pool

In [ ]:
oil = pd.read_csv("data/oil.csv")
stores = pd.read_csv("data/stores.csv")
holidays_events = pd.read_csv("data/holidays_events.csv")
transactions = pd.read_csv("data/transactions.csv")
items = pd.read_csv("data/items.csv")
mypath = "data/train/"
trainFileList = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

In [6]:
print(oil.isnull().sum())
oil = oil.fillna(method="pad")
oil = oil.fillna(value=93.14, limit=1)
oil.head()

date          0
dcoilwtico    0
dtype: int64


,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [52]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [53]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [54]:
transactions.head()

In [55]:
trainFileTest.head(
)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,22000000,2014-04-07,39,215352,38.0,False
1,22000001,2014-04-07,39,215356,1.0,False
2,22000002,2014-04-07,39,215370,4.0,False
3,22000003,2014-04-07,39,215896,11.0,False
4,22000004,2014-04-07,39,218728,4.0,False


In [7]:
def buildDataSet(mainTrain):
    # mainTrain = pd.merge(left=mainTrain, right=stores, how="left", left_on="store_nbr", right_on="store_nbr")
    # mainTrain = pd.merge(left=mainTrain, right=transactions, how="left", left_on=["date", "store_nbr"], right_on=["date", "store_nbr"])
    # mainTrain = pd.merge(left=mainTrain, right=oil, how="left", left_on=["date"], right_on=["date"])
    # mainTrain = pd.merge(left=mainTrain, right=items, how="left", left_on=["item_nbr"], right_on=["item_nbr"])
    mainTrain["date"] = mainTrain["date"].apply(lambda x: pd.to_datetime(x))
    mainTrain["month"] = mainTrain["date"].apply(lambda x: x.month)
    mainTrain["year"] = mainTrain["date"].apply(lambda x: x.year)
    mainTrain["day"] = mainTrain["date"].apply(lambda x: x.day)
    return mainTrain
num_partitions = 10 #number of partitions to split dataframe
num_cores = 14 #number of cores on your machine
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df
trainArray = parallelize_dataframe( pd.read_csv("data/train/output_%s.csv" %23),buildDataSet)

In [66]:
num =8


testArray = parallelize_dataframe(buildDataSet, pd.read_csv("data/train/output_%s.csv" %str(23+num)))
for x in range(24,23+num):
    print(x)
    trainArray += parallelize_dataframe(buildDataSet, pd.read_csv("data/train/output_%s.csv" %str(x)))

TypeError: object of type 'function' has no len()

In [41]:
store_nbr = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("store_nbr", hash_bucket_size=54), dimension=3)
onpromotion = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("onpromotion", hash_bucket_size=2), dimension=3)

0    4
1    4
2    4
3    4
4    4
5    4
6    4
7    4
8    4
9    4
Name: month, dtype: int64